In [1]:
import pandas as pd
import os
import time

In [2]:
MAIN_DIR = "/Volumes/Extreme SSD"
DATA_DIR = os.path.join(MAIN_DIR, "data")

## Feedback information
#### Dropping duplicate rows

In [3]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'feedbacks.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'feedbacks.pickle')

df = pd.read_pickle(data_file)
mem = df.memory_usage(deep = True).sum()
print(len(df), len(df.drop_duplicates()), len(df) - len(df.drop_duplicates()))
df = df.drop_duplicates()

2275492 2152745 122747


In [8]:
def get_day(day):
    day = day.str.split(' ').str[0]
    day = day.replace('today', '0')
    pd.to_numeric(day)
    return day

def to_dt(col):
    return pd.to_datetime(col, unit = 's', errors = 'ignore')

In [5]:
df['freshness'] = pd.to_numeric(get_day(df.freshness))
df = df\
    .assign(rating = pd.to_numeric(df.rating.str.get(0)),
            rtime  = df.stime - df.freshness * 86400)\
    .assign(rtime_dt = lambda df_copy: to_dt(df_copy.rtime))

In [6]:
print('memory usage before cleaning                    ', mem)
print('memory usage before memory usage optimalization ', df.memory_usage(deep = True).sum())

column_types = {
    'name'     : 'category',
    'stime'    : 'float32',
    'rating'   : pd.Int16Dtype(),
    'feedback' : 'object',
    'freshness': 'float32',
    'rtime'    : 'float32',
    'rtime_dt' : 'datetime64'
}

df = df.astype(column_types)

print('memory usage after memory usage optimalization  ', df.memory_usage(deep = True).sum())

memory usage before cleaning                     890426282
memory usage before memory usage optimalization  639579799
memory usage after memory usage optimalization   460772248


#### dropping duplicates

In scaped data, the same data is often scraped multiple times. To assure that feedbacks are not included twice duplicates will be dropped. Feedbacks can be uniquely identified by: 
- `name`
- `rating`
- `feedback`
- `rtime_dt`

In [7]:
# drop duplicate feedbacks from table 
subset = ['name', 'rating', 'feedback', 'rtime_dt']
print("before dropping duplicates: {0}\nafter dropping duplicates:  {1}\n\n{2} duplicate cases were dropped".format(len(df), len(df.drop_duplicates(subset = subset)), len(df) - len(df.drop_duplicates(subset = subset))))

df = df.drop_duplicates(subset = subset)
df = df.reset_index()

before dropping duplicates: 2152745
after dropping duplicates:  2145909

6836 duplicate cases were dropped


In [8]:
df.to_pickle(out_file)

## Vendor information


In [9]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'vendors.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'vendors.pickle')

df = pd.read_pickle(data_file)
mem = df.memory_usage(deep = True).sum()

In [6]:
def date_to_num(col):
    """
    This function returns a numeric value for valid date strings
    If the date is today, a zero-value is returned
    Otherwise a Null value is returned 
    """
    if isinstance(col, str):
        if 'today' not in col:
            trans_dict = {
                'year'  : 365.25,
                'month' : 365.25/12,
                'day'   : 1,
                'hour'  : 1/24
            }

            value = float(col.split(' ')[0])
            unit = col.split(' ')[1]
            for key in trans_dict.keys():
                if unit in key:
                    unit = unit.replace(key, str(trans_dict[key]))
            unit = float(unit)

            return value * unit
        
        elif 'today' in col:
            return 0
        
        else:
            return None

In [11]:
df = df\
    .assign(
        ctime     = df.ctime.str.replace("about ", "").str.replace('s', '').str.replace(' ago', ''),
        otime     = df.otime.str.replace("about ", "").str.replace('s', '').str.replace(' ago', ''))\
    .assign(
        ctime_num = lambda df_copy: df_copy['ctime'].apply(date_to_num),
        otime_num = lambda df_copy: df_copy['otime'].apply(date_to_num))\
    .assign(
        otime_dt  = lambda df_copy: to_dt(df_copy.stime - df_copy.otime_num * 86400),
        ctime_dt  = lambda df_copy: to_dt(df_copy.stime - df_copy.ctime_num * 86400),
        stime_dt  = to_dt(df.stime),
        score     = df.score.replace('NEW VENDOR', 0).apply(pd.to_numeric))\
    .assign(
        score     = lambda df_copy: df_copy.score.mask(df_copy.score > 100)
    )

In [12]:
# drop duplicate feedbacks from table 
df['flag'] = df.ctime_dt.dt.strftime('%d%m%Y').astype('category')
subset = ['name', 'flag', 'location', 'area']
print("before dropping duplicates: {0}\nafter dropping duplicates:   {1}\n\n{2} duplicate cases were dropped".format(len(df), len(df.drop_duplicates(subset = subset)), len(df) - len(df.drop_duplicates(subset = subset))))
df = df.drop_duplicates(subset = subset)

before dropping duplicates: 12242
after dropping duplicates:  9377

2865 duplicate cases were dropped


In [13]:
columns = ['name', 'stime', 'stime_dt', 'score', 'ctime', 'ctime_num', 'ctime_dt', 
           'otime', 'otime_num', 'otime_dt', 'location', 'area']

df = df.reindex(columns, axis=1)
df = df.reset_index()

In [14]:
print('memory usage before cleaning                    ', mem)
print('memory usage before memory usage optimalization ', df.memory_usage(deep = True).sum())

column_types = {
    'name'     : 'category',
    'stime'    : 'float32',
    'stime_dt' : 'datetime64',
    'score'    : pd.Int16Dtype(),
    'ctime'    : 'category', 
    'ctime_num': 'float32', 
    'ctime_dt' : 'datetime64', 
    'otime'    : 'category', 
    'otime_num': 'float32', 
    'otime_dt' : 'datetime64', 
    'location' : 'category', 
    'area'     : 'category'
}

df = df.astype(column_types)

print('memory usage after memory usage optimalization   ', df.memory_usage(deep = True).sum())

memory usage before cleaning                     5708163
memory usage before memory usage optimalization  3695920
memory usage after memory usage optimalization    640278


In [15]:
df.to_pickle(out_file)

## Item Information

In [3]:
data_file = os.path.join(MAIN_DIR, 'data', 'parsed', 'silkroad2', 'items.pickle')
out_file  = os.path.join(MAIN_DIR, 'data', 'final', 'silkroad2', 'items.pickle')

df = pd.read_pickle(data_file)
print(len(df), len(df.drop_duplicates()), len(df) - len(df.drop_duplicates()))
df = df.drop_duplicates()

22003618 22003618 0


In [4]:
def clean_freshness(col):
    #assert that freshness is not missing
    new_col = []
    for i in col:
        if not isinstance(i, (float, int)) and 'day' in i and len(i) < 10:
            new_col.append(i)
        else:
            new_col.append(None)
    
    return pd.Series(new_col)

# mem = df.memory_usage(deep = True).sum()

In [9]:
df = df\
    .assign(
        vendor = df.vendor.str.replace('\\n', ''),
        freshness = clean_freshness(df.freshness).\
            str.replace('s', '')\
            .apply(date_to_num),
        rating = df.rating.str.get(0),
        price = df.price.str[1:],
        stime_str = df.stime_dt)\
    .assign(
        rtime = lambda df_copy: df_copy.stime - df_copy.freshness *86400)\
    .assign(
        rtime_dt = lambda df_copy: to_dt(df_copy.rtime),
        stime_dt = lambda df_copy: to_dt(df_copy.stime))

In [11]:
# print('memory usage before cleaning', mem)
# print('memory usage before memory usage optimalization', df.memory_usage(deep = True).sum())

column_types = {
    'vendor'   : 'category',
    'stime'    : 'float32',
    'stime_dt' : 'datetime64',
    'stime_str': 'category',
    'rating'   : 'float16',
    'feedback' : 'object',
    'item'     : 'object',
    'price'    : 'float32',
    'freshness': 'float32',
    'rtime'    : 'float32',
    'rtime_dt' : 'datetime64',
    'loc'      : 'category',
    'area'     : 'category'
}

df = df.astype(column_types)

# downcast integer variables
df = df\
    .assign(
        freshness = pd.Series(df.freshness, dtype=pd.Int16Dtype()),
        rating = pd.Series(df.rating, dtype=pd.Int16Dtype()))

print('memory usage after memory usage optimalization', df.memory_usage(deep = True).sum())

memory usage after memory usage optimalization 5984469841


In [12]:
# drop duplicate feedbacks from table 
df['flag'] = df.rtime_dt.dt.strftime('%d%m%Y').astype('category')
subset = ['vendor', 'flag', 'loc', 'feedback', 'rating']
print("before dropping duplicates: {0}\nafter dropping duplicates:  {1}\n\n{2} duplicate cases were dropped".format(len(df), len(df.drop_duplicates(subset = subset)), len(df) - len(df.drop_duplicates(subset = subset))))
df = df.drop_duplicates(subset = subset)
df['location'] = df['loc']

before dropping duplicates: 22003618
after dropping duplicates:  1362588

20641030 duplicate cases were dropped


In [13]:
columns = ['vendor', 'stime', 'stime_dt', 'stime_str', 'rating', 'feedback', 'item', 'price',
         'freshness', 'rtime', 'rtime_dt', 'location', 'area']

df = df.reindex(columns, axis=1)
df = df.reset_index(drop = True)

In [14]:
df.sample(5)

,vendor,stime,stime_dt,stime_str,rating,feedback,item,price,freshness,rtime,rtime_dt,location,area
213552,haizenberg,1.402430e+09,2014-06-10 20:01:36,2014-04-12,5,Hope it shows up,"1G MDMA 87% Purity , QUALITY GUARANTEED",0.066378,5,1.401998e+09,2014-06-05 20:01:36,Czech Republic,Worldwide
113765,mr whippy pharmaceuticals,1.402430e+09,2014-06-10 19:57:45,2014-03-10,5,"Received the next day, product as descripted. ...",0.5g DMT (NN-DMT),0.080224,51,1.398024e+09,2014-04-20 19:57:45,United Kingdom,United Kingdom
1202771,trava!95,1.414109e+09,2014-10-24 00:06:26,2014-10-24,5,As always ridiculously fast delivery and the s...,14g STINKY BUD,0.665160,2,1.413936e+09,2014-10-22 00:06:26,United Kingdom,United Kingdom
3881,ItalianMafiaBrussels,1.402429e+09,2014-06-10 19:32:47,2013-12-20,5,++++,250g Pure MDMA 80%+ Pure Best Quality/Price,7.628963,0,1.402429e+09,2014-06-10 19:32:47,Belgium,Worldwide
779978,DeweyDylan93,1.409305e+09,2014-08-29 09:35:54,2014-08-30,5,Fast shipping. some pills broken and crumbled ...,Big Hawaiian Seeds x 5,0.039533,262,1.386668e+09,2013-12-10 09:35:54,United States,United States


In [15]:
df.to_pickle(out_file)